In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


data = pd.read_csv("data368", error_bad_lines=False)

b'Skipping line 22321: expected 15 fields, saw 16\nSkipping line 29629: expected 15 fields, saw 17\nSkipping line 37133: expected 15 fields, saw 21\nSkipping line 51819: expected 15 fields, saw 18\nSkipping line 59214: expected 15 fields, saw 17\n'
b'Skipping line 73833: expected 15 fields, saw 18\nSkipping line 88670: expected 15 fields, saw 27\nSkipping line 118133: expected 15 fields, saw 25\n'
b'Skipping line 147448: expected 15 fields, saw 21\nSkipping line 162190: expected 15 fields, saw 25\nSkipping line 184339: expected 15 fields, saw 22\n'
b'Skipping line 198761: expected 15 fields, saw 24\nSkipping line 220651: expected 15 fields, saw 18\nSkipping line 235260: expected 15 fields, saw 27\n'
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2,3,4,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
data.drop('Unnamed: 0', axis = 1, inplace=True)

In [0]:
data.head()

,labels,Ave_D,distance,clouds,trip,pressure,humidity,tempSRQT,rainSRQT,windSRQT,price_noSM,category,hour_of_day,day_of_week
0,0.0,0.589588,0.44,0.29,Haymarket Square_North Station,1022.25,0.76,6.20161,0.0,2.7712812921102037,5,Shared,9.0,Sunday
1,1.0,1.26586,1.33,0.29,Haymarket Square_Beacon Hill,1022.25,0.76,6.20161,0.0,2.7712812921102037,7,Regular,9.0,Sunday
2,0.0,0.589588,0.53,0.29,Haymarket Square_North Station,1022.25,0.76,6.20161,0.0,2.7712812921102037,27.5,LuxuryXL,9.0,Sunday
3,2.0,2.28312,2.36,0.29,Haymarket Square_Back Bay,1022.25,0.76,6.20161,0.0,2.7712812921102037,7,Shared,9.0,Sunday
4,3.0,0.697945,0.62,0.29,Haymarket Square_West End,1022.25,0.76,6.20161,0.0,2.7712812921102037,10.5,XL,9.0,Sunday


In [0]:
X1 = data[data.trip == "Financial District_South Station"]
X2 = data[data.trip == "Theatre District_South Station"]

In [0]:
X1.shape, X2.shape

((9967, 14), (9255, 14))

In [0]:
len(X1[X1.category == "Regular"]), len(X2[X2.category == "Regular"]), 

(1970, 1846)

In [0]:
d = X1[['distance', 'price_noSM']]
s = X2[['distance', 'price_noSM']]
print(d.price_noSM.mean(), s.price_noSM.mean())

# **K-means clustering**

In [0]:
X_total = pd.concat([X1, X2], axis=0)

In [0]:
X_total = X_total[X_total.category=="Regular"]

In [0]:
for_cluster = X_total[['distance', 'price_noSM']]

In [0]:
from sklearn.cluster import KMeans

km = KMeans(
    n_clusters=2,
    n_init=10,
    tol=1e-04, random_state=1280
)
y_km = km.fit_predict(for_cluster)

In [0]:
km.cluster_centers_

array([[0.49714828, 6.92721555],
       [0.53648718, 9.79333333]])

In [0]:
cluster2 = for_cluster.to_numpy()

In [0]:
clusters_centroids=dict()
clusters_radii= dict()

for cluster in list(set(y_km)):

    clusters_centroids[cluster]=list(zip(km.cluster_centers_[:, 0],km.cluster_centers_[:,1]))[cluster]
    clusters_radii[cluster] = max([np.linalg.norm(np.subtract(i,clusters_centroids[cluster])) for i in zip(cluster2[y_km == cluster, 0],cluster2[y_km == cluster, 1])])

UFuncTypeError: ignored

In [0]:
import matplotlib.patches as mpatches

fig, ax = plt.subplots(1, figsize=(18,16))
ax.yaxis.tick_right()
ax.invert_yaxis()

plt.figure(1, figsize=(18, 16))
plt.scatter(cluster2[:,0], cluster2[:,1], c=y_km, marker = 'o')
art = mpatches.Circle(clusters_centroids[0],clusters_radii[0], edgecolor='b',fill=False, label ="Financial District")
ax.add_patch(art)
art = mpatches.Circle(clusters_centroids[1],clusters_radii[1], edgecolor='r',fill=False, label ="Theatre District")
ax.add_patch(art)

plt.scatter(km.cluster_centers_[0,0], km.cluster_centers_[0,1], c='red', marker = "*", s=700)
plt.scatter(km.cluster_centers_[1,0], km.cluster_centers_[1,1], c='red', marker = "*", s=700)


plt.title("Price distribution vs distance travelling to South Station, Boston, US", fontsize= 30)
plt.xlabel("Distance (mile)", fontsize= 20)
plt.ylabel("Price USD", fontsize= 20)
plt.legend(loc='best', prop={'size':30})
plt.xlim([0:2])
plt.grid()
plt.show

SyntaxError: ignored

# **Linear regression stuff**

In [0]:
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import train_test_split

In [0]:
data = pd.get_dummies(data, columns=['trip', 'category', 'day_of_week'])

In [0]:
from sklearn.preprocessing import StandardScaler

def standarize(x_train, x_test, columns):
    scaler = StandardScaler()
    x_train[columns] = scaler.fit_transform(x_train[columns])
    x_test[columns] = scaler.transform(x_test[columns])
                        
    return x_train, x_test

In [0]:
X = data.drop('price_noSM', axis=1)
y = data['price_noSM']

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1248)

In [0]:
X_train, X_test = standarize(X_train, X_test, ['Ave_D','distance', 'clouds', 'pressure', 'rainSRQT','tempSRQT', 'windSRQT'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.py

# **Linear Regression with all features**

In [0]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
linreg.predict(X_test)

train_score = linreg.score(X_train, y_train)
test_score = linreg.score(X_test, y_test)

print(train_score, test_score)

0.8839581296884627 0.8824439743432253


# **LASSO with all features**

In [0]:
lasso = LassoCV()
lasso.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


LassoCV(alphas=None, copy_X=True, cv='warn', eps=0.001, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [0]:
train_score = lasso.score(X_train, y_train)
test_score = lasso.score(X_test, y_test)

print(train_score, test_score)

0.880977643938881 0.879553452516146


In [0]:
betas = pd.Series(lasso.coef_, index=X_train.columns)
betas = betas.append(pd.Series({"Intercept": lasso.intercept_}))
print([beta for beta in betas])

[0.0007436739843804795, 0.8353412297030339, 2.4301673283715943, -0.0015813724010331576, -0.0, -0.0, -0.0, -0.0, 0.004252759863424623, 0.0009123383708435466, -0.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0, 0.11063965207870122, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.0, -0.0, 6.10400221064187, 15.526749093491766, -5.698200213698931, -7.932676208921571, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 15.350453126120925]


In [0]:
X_train.head()

,labels,Ave_D,distance,clouds,pressure,humidity,tempSRQT,rainSRQT,windSRQT,hour_of_day,trip_Back Bay_Boston University,trip_Back Bay_Fenway,trip_Back Bay_Haymarket Square,trip_Back Bay_North End,trip_Back Bay_Northeastern University,trip_Back Bay_South Station,trip_Beacon Hill_Boston University,trip_Beacon Hill_Fenway,trip_Beacon Hill_Haymarket Square,trip_Beacon Hill_North End,trip_Beacon Hill_Northeastern University,trip_Beacon Hill_South Station,trip_Boston University_Back Bay,trip_Boston University_Beacon Hill,trip_Boston University_Financial District,trip_Boston University_North Station,trip_Boston University_Theatre District,trip_Boston University_West End,trip_Fenway_Back Bay,trip_Fenway_Beacon Hill,trip_Fenway_Financial District,trip_Fenway_North Station,trip_Fenway_Theatre District,trip_Fenway_West End,trip_Financial District_Boston University,trip_Financial District_Fenway,trip_Financial District_Haymarket Square,trip_Financial District_North End,trip_Financial District_Northeastern University,trip_Financial District_South Station,...,trip_North Station_Haymarket Square,trip_North Station_North End,trip_North Station_Northeastern University,trip_North Station_South Station,trip_Northeastern University_Back Bay,trip_Northeastern University_Beacon Hill,trip_Northeastern University_Financial District,trip_Northeastern University_North Station,trip_Northeastern University_Theatre District,trip_Northeastern University_West End,trip_South Station_Back Bay,trip_South Station_Beacon Hill,trip_South Station_Financial District,trip_South Station_North Station,trip_South Station_Theatre District,trip_South Station_West End,trip_Theatre District_Boston University,trip_Theatre District_Fenway,trip_Theatre District_Haymarket Square,trip_Theatre District_North End,trip_Theatre District_Northeastern University,trip_Theatre District_South Station,trip_West End_Boston University,trip_West End_Fenway,trip_West End_Haymarket Square,trip_West End_North End,trip_West End_Northeastern University,trip_West End_South Station,category_Luxury,category_LuxuryXL,category_Regular,category_Shared,category_XL,day_of_week_Friday,day_of_week_Monday,day_of_week_Saturday,day_of_week_Sunday,day_of_week_Thursday,day_of_week_Tuesday,day_of_week_Wednesday
89612,5,-1.062193,-1.056478,-0.407229,0.831200,0.71,1.348703,-0.316892,-0.234762,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0
189955,4,-0.706541,-0.510728,1.058602,-1.138055,0.93,0.638178,0.795990,0.889887,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0
290391,1,-0.871442,-1.126898,1.026736,-0.653396,0.89,0.749265,6.440877,1.374431,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0
177665,6,-0.836047,-0.475518,-1.522534,1.759446,0.77,-0.842217,-0.316892,-0.992951,14,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
527578,25,0.836901,0.739217,-0.725887,-0.819927,0.70,-0.130527,-0.316892,0.853336,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0


# **LASSO with reduced features**

In [0]:
newX_train = X_train[['Ave_D','distance', 'clouds', 'windSRQT', 'hour_of_day',
                     'trip_North End_Back Bay', 'category_Shared', 'category_Regular', 
                      'category_LuxuryXL', 'category_Luxury']]

In [0]:
newX_test = X_test[['Ave_D','distance', 'clouds', 'windSRQT', 'hour_of_day',
                     'trip_North End_Back Bay','category_Shared', 'category_Regular', 
                    'category_LuxuryXL','category_Luxury']]

In [0]:
lasso2 = LassoCV()
lasso2.fit(newX_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


LassoCV(alphas=None, copy_X=True, cv='warn', eps=0.001, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [0]:
train_score = lasso2.score(newX_train, y_train)
test_score = lasso2.score(newX_test, y_test)

print(train_score, test_score)

0.8812057950121471 0.8797450165922671


In [0]:
betas = pd.Series(lasso2.coef_, index=newX_train.columns)
betas = betas.append(pd.Series({"Intercept": lasso2.intercept_}))
print(betas)

Ave_D                       0.844522
distance                    2.430969
clouds                     -0.016447
windSRQT                    0.017789
hour_of_day                 0.001243
trip_North End_Back Bay     0.964577
category_Shared            -7.996832
category_Regular           -5.762876
category_LuxuryXL          15.588913
category_Luxury             6.164917
Intercept                  15.348214
dtype: float64


# **Linear Regression with reduced features**

In [0]:
linreg2 = LinearRegression()
linreg2.fit(newX_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
train_score = linreg2.score(newX_train, y_train)
test_score = linreg2.score(newX_test, y_test)

print(train_score, test_score)

0.8812158489519033 0.8797470069241341


In [0]:
df = pd.read_csv('https://drive.google.com/uc?export=download&id=167Nqjl4Q9D-bw72qJBrAm8ZYL-ANIh2Q')

In [0]:
df['cab_type'] = df['cab_type'].map({'Uber':1, 'Lyft':-1})
df.groupby('cab_type').size()

cab_type
-1    307408
 1    385663
dtype: int64

In [0]:
cab_type = df['cab_type']

In [0]:
newX_train.insert(1, 'cab_type', cab_type)
X_train.insert(1, 'cab_type', cab_type)
newX_test.insert(1, 'cab_type', cab_type)
X_test.insert(1, 'cab_type', cab_type)


## **Linear Regression and Lasso for Uber with the reduced dataset**

In [0]:
data1 = newX_train.copy()

In [0]:
data1.insert(1, 'Price', y_train)

In [0]:
data_uber = data1[data1.cab_type == 1]

In [0]:
X_uber = data_uber.drop(['Price', 'cab_type'], axis=1)
y_uber = data_uber['Price']

In [0]:
X_trainU, X_testU, y_trainU, y_testU = train_test_split(X_uber, y_uber, test_size=0.33, random_state=1248)

In [0]:
linregU = LinearRegression()
linregU.fit(X_trainU, y_trainU)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
train_score = linregU.score(X_trainU, y_trainU)
test_score = linregU.score(X_testU, y_testU)

print(train_score, test_score)

0.8929456537529769 0.890430232517606


In [0]:
linsU = pd.Series(linregU.coef_, index=X_trainU.columns)
linsU = linsU.append(pd.Series({"Intercept": linregU.intercept_}))
print(linsU)

Ave_D                       0.886751
distance                    2.415381
clouds                     -0.021042
windSRQT                    0.020496
hour_of_day                 0.001700
trip_North End_Back Bay     1.127188
category_Shared            -8.070115
category_Regular           -5.801647
category_LuxuryXL          15.756592
category_Luxury             6.225059
Intercept                  15.450130
dtype: float64


In [0]:
lassoU = LassoCV()
lassoU.fit(X_trainU, y_trainU)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


LassoCV(alphas=None, copy_X=True, cv='warn', eps=0.001, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [0]:
train_score = lassoU.score(X_trainU, y_trainU)
test_score = lassoU.score(X_testU, y_testU)

print(train_score, test_score)

0.8929356629235685 0.8904063382224408


In [0]:
betasU = pd.Series(lassoU.coef_, index=X_trainU.columns)
betasU = betasU.append(pd.Series({"Intercept": lassoU.intercept_}))
print(betasU)

Ave_D                       0.896557
distance                    2.404807
clouds                     -0.017298
windSRQT                    0.016904
hour_of_day                 0.001640
trip_North End_Back Bay     0.909422
category_Shared            -8.053996
category_Regular           -5.785423
category_LuxuryXL          15.740319
category_Luxury             6.209148
Intercept                  15.454095
dtype: float64


# **Linear Regression and Lasso for Lyft with the reduced dataset**

In [0]:
data_lyft = data1[data1.cab_type == -1]

In [0]:
X_lyft = data_lyft.drop(['Price', 'cab_type'], axis=1)
y_lyft = data_lyft['Price']

In [0]:
X_trainL, X_testL, y_trainL, y_testL = train_test_split(X_lyft, y_lyft, test_size=0.33, random_state=1248)

In [0]:
linregL = LinearRegression()
linregL.fit(X_trainL, y_trainL)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
train_score = linregL.score(X_trainL, y_trainL)
test_score = linregL.score(X_testL, y_testL)

print(train_score, test_score)

0.8683304709452798 0.8676601442914642


In [0]:
linsL = pd.Series(linregL.coef_, index=X_trainL.columns)
linsL = linsL.append(pd.Series({"Intercept": linregL.intercept_}))
print(linsL)

Ave_D                       0.848752
distance                    2.378268
clouds                     -0.022719
windSRQT                    0.025472
hour_of_day                 0.001787
trip_North End_Back Bay     1.145477
category_Shared            -7.911941
category_Regular           -5.720384
category_LuxuryXL          15.410329
category_Luxury             6.118676
Intercept                  15.177911
dtype: float64


In [0]:
lassoL = LassoCV()
lassoL.fit(X_trainL, y_trainL)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


LassoCV(alphas=None, copy_X=True, cv='warn', eps=0.001, fit_intercept=True,
        max_iter=1000, n_alphas=100, n_jobs=None, normalize=False,
        positive=False, precompute='auto', random_state=None,
        selection='cyclic', tol=0.0001, verbose=False)

In [0]:
train_score = lassoL.score(X_trainL, y_trainL)
test_score = lassoL.score(X_testL, y_testL)

print(train_score, test_score)

0.8683205118375668 0.8676248584405536


In [0]:
betasL = pd.Series(lassoL.coef_, index=X_trainL.columns)
betasL = betasL.append(pd.Series({"Intercept": lassoL.intercept_}))
print(betasL)

Ave_D                       0.860491
distance                    2.365771
clouds                     -0.019048
windSRQT                    0.022330
hour_of_day                 0.001683
trip_North End_Back Bay     0.921326
category_Shared            -7.896823
category_Regular           -5.704588
category_LuxuryXL          15.394343
category_Luxury             6.103033
Intercept                  15.182401
dtype: float64
